# `dgaf` tests

`dgaf` tests three major aspects of the project:

1. inference - `dgaf` infers project configurations from the conformation of documents in folders and repositories.
2. configuration - `dgaf` generates configuration files for projects in different file formats.
3. verify - `dgaf` verifies that the configuration files are compliant with their corresponding tools.

[there are more tests in the documentation](docs/readme.md)

    [tool.dgaf]
    id = "<url>"
    infer = False
    build = "flit"
    install = "conda"
    develop = True
    docs = "jupyter-book"
    lint = "flakehell/pre-commit"
    name = "tonyfast"
    email = "tony.fast@gmail.com"
    venv = True

## test functions

`inference` tests the interactive `dgaf` api for its ability to generate the proper configurations for the `project`.

In [1]:
    def inference(pytester, project):
        assert project.is_flit()
        assert project.get_name() == "my_idea"
        assert project.get_description() == "my projects docstring"
        assert project.get_version() == "0.0.1"
        assert project.get_requires() == ["pandas"]

`configuration` tests the ability for the `project` to configure itself. the primary intent of `dgaf` is distribution literate programs as literature and software. below we test the abilities to:

* generate pyproject.toml file for building the project.
* generate table of contents and documentation configuration.
* configure the linters and formatters

a lot of information can be encoded in a the `pyproject.toml` file. `dgaf` configures `flakehell, pytest and flit` in these tests.

In [9]:
    def configuration(pytester, project):
        project.add("python")
        assert (project/ "pyproject.toml").exists()
        
        project.add("docs")
        assert (project/ "docs/_toc.yml").exists() and (project/ "docs/_config.yml").exists()
        
        project.add("lint")
        assert (project / '.pre-commit-config.yaml').exists()        

`verify` checks the ability of different tools to use the generated configurations.

the conventions we choose have either implicit/explicit schema and uses consumers of those schema to verify the configuration has the appropriate forms. we do not explicitly verify schema with `dgaf`.

In [10]:
    def verify(pytester, project):
        import dgaf
        pyproject = dgaf.File(project/ "pyproject.toml").load()
        print(pyproject)
        tool = pyproject.get("tool", {})
        assert all(x in tool for x in "flit pytest".split())
        assert not pytester.run(*"flit build --format sdist".split()).ret

## the test objects

`build` creates a test directory from nested dictionaries.

In [11]:
    import pathlib, pytest
    def build(pytester, object, where=None):
        for key, value in object.items():
            if isinstance(value, str):
                file = (where or pathlib.Path()) / key
                if where:
                    where.mkdir(exist_ok=True, parents=True)
                if file.suffix == ".ipynb":
                    import nbformat
                    value = nbformat.v4.writes(nbformat.v4.new_notebook(cells=[nbformat.v4.new_code_cell(contents)]))
                pytester.makefile(file.suffix, **{
                    str(file.with_suffix("")): value
                })
            elif isinstance(value, dict):
                build(pytester, value, where=(where or pathlib.Path())/key)

for this test document we'll consider a simple project with the contents below. in the `contents`, we need to explicitly provide a docstring and version to cooperate with `flit`s model.

In [12]:
    contents = """'''my projects docstring'''
    __version__ = "0.0.1"
    
    import pandas
    """

it allows different layouts, like `python_layouts` to be used as test input.

In [13]:
    python_layouts = [{
        "my_idea.py": contents
    }, dict(
        my_idea={
            "__init__.py": contents
        }
    ), dict(
        src=dict(
            my_idea={
                "__init__.py": contents
            }
        )
    ), {
        "my_idea.ipynb": contents
    }]

In [14]:
    @pytest.mark.parametrize("layout", python_layouts)
    def test_python(pytester, layout):
        import dgaf
        build(pytester, layout)
        project = dgaf.Project(pytester.path)
        configuration(pytester, project)
        project = dgaf.Project(pytester.path)
        assert project.is_flit()
        inference(pytester, project)
        verify(pytester, project)

## test the `src.dgaf` directly.

In [1]:
    import dgaf

'dgaf'

In [15]:
    def test_dgaf_project():
        from src import dgaf        
        self = project = dgaf.Project()
        assert project.get_name() == "dgaf" == dgaf.Project("src").get_name()
        assert project.is_flit()
        assert project.get_description()
        requires = ["nox", "typer"]
        print(project.get_requires())
        
        assert project.get_requires() == ["nox", "typer"]
        assert project.get_url()
        assert "@" in project.get_email()
        assert 'docs/_build/' in project.get_exclude()